In [1]:
import math
import numpy as np
import os
import sys
import pandas as pd
import pickle

import torch
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import multiprocessing
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, BertForPreTraining, BertModel, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification, AutoConfig, AutoModel
from datasets import load_dataset

# Initialize data and pattern

In [2]:
def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", return_tensors='pt', truncation=True)

MODEL = f"cardiffnlp/twitter-roberta-base-dec2021-tweet-topic-single-all"

tokenizer = AutoTokenizer.from_pretrained(MODEL, model_max_length=32)
_, test_ds = load_dataset("cardiffnlp/tweet_topic_single", split=['train_coling2022', 'test_coling2022[:1000]'])

encoded_test = test_ds.map(tokenize, batched=True)
encoded_test.set_format("pt", columns=["input_ids"], output_all_columns=True)

Found cached dataset tweet_topic_single (/home/minhvu/.cache/huggingface/datasets/cardiffnlp___tweet_topic_single/tweet_topic_single/1.0.4/832eaa087889d9f4bc549869b44e0acb85a78364dfb3d2bc0bdf23a7224cf2ce)


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter 'function'=<function tokenize at 0x2b23c31425e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Pattern

In [3]:
sentence_for_testing = "This film was probably inspired by Harry. I think it is fanstastic."

sen1 = torch.tensor(tokenizer.encode(sentence_for_testing)).unsqueeze(0)
pattern_id = sen1[0][7] # the pattern is "Harry"

### Remove pattern from the dataset

In [4]:
test_input_ids = torch.where(encoded_test['input_ids'] == pattern_id, 0, encoded_test['input_ids'])

### Get the embedding of the pattern in the model

In [5]:
model = AutoModel.from_pretrained(MODEL, output_hidden_states = True)
x_test = model(test_input_ids)['hidden_states'][0]
sen1_emb = model(torch.tensor(tokenizer.encode(sentence_for_testing)).unsqueeze(0))['hidden_states'][0]
pattern = sen1_emb[0][7]

pattern = pattern.detach()
x_test = x_test.detach()
x_test = x_test.reshape(x_test.shape[0] * x_test.shape[1], x_test.shape[2])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-dec2021-tweet-topic-single-all were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-dec2021-tweet-topic-single-all and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stre

In [6]:
# sen1_emb = model(torch.tensor(tokenizer.encode(sentence_for_testing)).unsqueeze(0))['hidden_states'][0]
# pattern = sen1_emb[0][7]

# LDP

In [7]:
NUM_PROCESS = 70
from tqdm import tqdm
import time
l = 20
m = 2
r = 768

def unpacking_apply_along_axis(all_args):
    (func1d, axis, arr, args, kwargs) = all_args
    
    """
    Like numpy.apply_along_axis(), but with arguments in a tuple
    instead.

    This function is useful with multiprocessing.Pool().map(): (1)
    map() only handles functions that take a single argument, and (2)
    this function can generally be imported from a module, as required
    by map().
    """
    return np.apply_along_axis(func1d, axis, arr, *args, **kwargs)

def parallel_apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """
    Like numpy.apply_along_axis(), but takes advantage of multiple
    cores.
    """        
    # Effective axis where apply_along_axis() will be applied by each
    # worker (any non-zero axis number would work, so as to allow the use
    # of `np.array_split()`, which is only done on axis 0):
    effective_axis = 1 if axis == 0 else axis
    if effective_axis != axis:
        arr = arr.swapaxes(axis, effective_axis)

    # Chunks for the mapping (only a few chunks):
    chunks = [(func1d, effective_axis, sub_arr, args, kwargs)
              for sub_arr in np.array_split(arr, NUM_PROCESS)]

    pool = multiprocessing.Pool(processes=NUM_PROCESS)
    individual_results = pool.map(unpacking_apply_along_axis, chunks)
    
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)

def parallel_matrix_operation(func, arr):
    chunks = np.array_split(arr, NUM_PROCESS)
    
    
    pool = multiprocessing.Pool(processes=NUM_PROCESS)
    individual_results = pool.map(func, chunks)
    
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)

def float_to_binary(x, m, n):
    x_abs = np.abs(x)
    x_scaled = round(x_abs * 2 ** n)
    res = '{:0{}b}'.format(x_scaled, m + n)
    if x >= 0:
        res = '0' + res
    else:
        res = '1' + res
    return res

# binary to float
def binary_to_float(bstr, m, n):
    sign = bstr[0]
#     print(int(sign))
    bs = bstr[1:]
    res = int(bs, 2) / 2 ** n
    if int(sign) == 49:
        res = -1 * res
    return res

def string_to_int(a):
    bit_str = "".join(x for x in a)
    return np.array(list(bit_str)).astype(int)


def join_string(a, num_bit=l, num_feat=r):
    res = np.empty(num_feat, dtype="S20")
    # res = []
    for i in range(num_feat):
        # res.append("".join(str(x) for x in a[i*l:(i+1)*l]))
        res[i] = "".join(str(x) for x in a[i*l:(i+1)*l])
    return res


def float_bin(x):
    return float_to_binary(x, m, l-m-1)
    

def bin_float(x):
    return binary_to_float(x, m, l-m-1)

In [8]:
def BitRand(sample_feature_arr, eps=10.0, l=20, m=2):

    r = sample_feature_arr.shape[1]
    
    float_to_binary_vec = np.vectorize(float_bin)
    binary_to_float_vec = np.vectorize(bin_float)

    feat_tmp = parallel_matrix_operation(float_to_binary_vec, sample_feature_arr)
    feat = parallel_apply_along_axis(string_to_int, axis=1, arr=feat_tmp)
    
#     return feat_tmp, feat

    rl = r * l
    sum_ = 0
    for k in range(l):
        sum_ += np.exp(2 * eps*k /l)
    alpha = np.sqrt((eps + rl) /( 2*r *sum_ ))
    index_matrix = np.array(range(l))
    index_matrix = np.tile(index_matrix, (sample_feature_arr.shape[0], r))
    p =  1/(1+alpha * np.exp(index_matrix*eps/l) )
    p_temp = np.random.rand(p.shape[0], p.shape[1])
    perturb = (p_temp > p).astype(int)
    

    perturb_feat = (perturb + feat)%2
    perturb_feat = parallel_apply_along_axis(join_string, axis=1, arr=perturb_feat)
    # print(perturb_feat)
    
#     return perturb_feat, perturb, feat
    
    return torch.tensor(parallel_matrix_operation(binary_to_float_vec, perturb_feat), dtype=torch.float)

In [9]:
def BitRand_1(sample_feature_arr, eps, l=20, m=2, r=768):
    float_bin_2 = lambda x: float_to_binary(x, m, l-m-1)
    float_to_binary_vec_2 = np.vectorize(float_bin_2)
    bin_float_2 = lambda x: binary_to_float(x, m, l-m-1)
    binary_to_float_vec_2 = np.vectorize(bin_float_2)

    feat_tmp = float_to_binary_vec_2(sample_feature_arr)
    feat = np.apply_along_axis(string_to_int, axis=1, arr=feat_tmp)
    sum_ = 0
    for k in range(l):
        sum_ += np.exp(2 * eps*k /l)
    alpha = np.sqrt((eps + r*l) /( 2*r *sum_ ))

    index_matrix = np.array(range(l))
    index_matrix = np.tile(index_matrix, (sample_feature_arr.shape[0], r))
    p =  1/(1+alpha * np.exp(index_matrix*eps/l) )
    p_temp = np.random.rand(p.shape[0], p.shape[1])
    perturb = (p_temp > p).astype(int)
    perturb_feat = (perturb + feat)%2
    perturb_feat = np.apply_along_axis(join_string, axis=1, arr=perturb_feat)
    perturb_feat = binary_to_float_vec_2(perturb_feat)

    return torch.squeeze(torch.tensor(perturb_feat, dtype=torch.float))#.cuda()

In [10]:
def OME(sample_feature_arr, eps=10.0, l=10, m=5):
    r = sample_feature_arr.shape[1]
    
    float_to_binary_vec = np.vectorize(float_bin)
    binary_to_float_vec = np.vectorize(bin_float)

    feat_tmp = parallel_matrix_operation(float_to_binary_vec, sample_feature_arr)
    feat = parallel_apply_along_axis(string_to_int, axis=1, arr=feat_tmp)

    rl = r * l
    alpha_ome = 100
    index_matrix_1 = np.array([alpha_ome / (1+ alpha_ome), 1/ (1+alpha_ome**3)]*int(l/2)) # np.array(range(l))
    index_matrix_0 = np.array([ (alpha_ome * np.exp(eps/rl)) /(1 + alpha_ome* np.exp(eps/rl))]*int(l) )
    p_1 = np.tile(index_matrix_1, (sample_feature_arr.shape[0], r))
    p_0 = np.tile(index_matrix_0, (sample_feature_arr.shape[0], r))

    p_temp = np.random.rand(p_0.shape[0], p_0.shape[1])
    perturb_0 = (p_temp > p_0).astype(int)
    perturb_1 = (p_temp > p_1).astype(int)

    perturb_feat = np.array(torch.where(torch.tensor(feat)>0, torch.tensor((perturb_1 + feat)%2), torch.tensor((perturb_0 + feat)%2)) )
    perturb_feat = parallel_apply_along_axis(join_string, axis=1, arr=perturb_feat)

    return torch.tensor(parallel_matrix_operation(binary_to_float_vec, perturb_feat), dtype=torch.float)

In [11]:
def OME_1(sample_feature_arr, eps=10.0, l=20, m=2):
    
    float_bin_2 = lambda x: float_to_binary(x, m, l-m-1)
    float_to_binary_vec_2 = np.vectorize(float_bin_2)
    bin_float_2 = lambda x: binary_to_float(x, m, l-m-1)
    binary_to_float_vec_2 = np.vectorize(bin_float_2)

    r = sample_feature_arr.shape[1]
    
    float_to_binary_vec = np.vectorize(float_bin)
    binary_to_float_vec = np.vectorize(bin_float)

    feat_tmp = float_to_binary_vec_2(sample_feature_arr)
    feat = np.apply_along_axis(string_to_int, axis=1, arr=feat_tmp)

    rl = r * l
    alpha_ome = 100
    index_matrix_1 = np.array([alpha_ome / (1+ alpha_ome), 1/ (1+alpha_ome**3)]*int(l/2)) # np.array(range(l))
    index_matrix_0 = np.array([ (alpha_ome * np.exp(eps/rl)) /(1 + alpha_ome* np.exp(eps/rl))]*int(l) )
    p_1 = np.tile(index_matrix_1, (sample_feature_arr.shape[0], r))
    p_0 = np.tile(index_matrix_0, (sample_feature_arr.shape[0], r))

    p_temp = np.random.rand(p_0.shape[0], p_0.shape[1])
    perturb_0 = (p_temp > p_0).astype(int)
    perturb_1 = (p_temp > p_1).astype(int)

    perturb_feat = np.array(torch.where(torch.tensor(feat)>0, torch.tensor((perturb_1 + feat)%2), torch.tensor((perturb_0 + feat)%2)) )
    perturb_feat = np.apply_along_axis(join_string, axis=1, arr=perturb_feat)

    perturb_feat = binary_to_float_vec_2(perturb_feat)
    return torch.squeeze(torch.tensor(perturb_feat, dtype=torch.float))#.cuda()

# The FC adversary

In [12]:
class Classifier(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(n_inputs, 2*n_inputs)
        self.fc2 = nn.Linear(2*n_inputs, 1)
        
    def adv_weights(self, target, tau, delta = 0):
        K = torch.eye(self.fc1.in_features)
        K = torch.cat((K, -K), 0)
        self.fc1.weight.data = K
        self.fc1.bias.data = torch.cat((-target - delta, target - delta), 0)
        
        self.fc2.weight.data[0] = -torch.ones(self.fc2.in_features)
        self.fc2.bias.data[0] = tau
        

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        fc2 = self.fc2(x)
        x = F.relu(fc2)
        return x, fc2

# Setting the security games

In [13]:
def task_tpr(i):
    np.random.seed(int((time.time()+i*1000)))
    x_test_threat = torch.cat((pattern.unsqueeze(0), x_test[np.random.randint(0, x_test.shape[0], D-1)]))
#     x_test_threat = OME_1(x_test_threat, eps, l=l, m=m) # change this for OME
    x_test_threat = BitRand_1(x_test_threat, eps, l=l, m=m)
    return x_test_threat

def task_tnr(i):
    np.random.seed(int((time.time()+i*1000)))
    x_test_threat = x_test[np.random.randint(0, x_test.shape[0], D)]
#     x_test_threat = OME_1(x_test_threat, eps, l=l, m=m) # change this for OME
    x_test_threat = BitRand_1(x_test_threat, eps, l=l, m=m)
    return x_test_threat

In [14]:
torch.multiprocessing.set_sharing_strategy('file_system')

# Find Tau

In [18]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [19]:
D = 10*32 # batch size
times = 200 # num of runs

model = Classifier(x_test.shape[1], 2)
model.adv_weights(pattern, 0.0)

# eps_taus = np.arange(0.5,7.1,0.5) # results in the paper
eps_taus = np.arange(6,7,1) # test for one run

In [20]:
taus = []
for eps_tau in eps_taus:
    
    eps = eps_tau
    print(f'eps={eps} --------------')
    
    torch.set_num_threads(1) # Required for multiprocessing 
    with multiprocessing.Pool(processes=NUM_PROCESS) as pool:
        x_tpr = list(tqdm(pool.imap_unordered(task_tpr, range(times), chunksize=5), total=times))
        x_tnr = list(tqdm(pool.imap_unordered(task_tnr, range(times), chunksize=5), total=times))

    torch.set_num_threads(72) # Required for multiprocessing 
    
    tmp_tau = 0
    for x in tqdm(x_tpr):
        out = model(x)[1]
        out_pattern = out[0]
        out_base = torch.mean(out[1:])
        tmp_tau = tmp_tau + (out_pattern + out_base)/2 
    tmp_tau = tmp_tau/len(x_tpr)
    taus.append(tmp_tau)
    

eps=6 --------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 87.54it/s]


# Evaluation

In [21]:
advs = []
tprs = []
tnrs = []

runs = 1
# runs = 10 # recommend number for good statistics 
games = 100
times = runs*games

# log_filename = f'API/twitter-res/Twitter_FC_OME_10.pkl'
log_filename = 'API/twitter-res/Twitter_FC_BitRand_10.pkl' 

result_log = pd.DataFrame()

In [22]:
for i in range(len(eps_taus)):
    
    eps = eps_taus[i]
    print(f'eps={eps} --------------------------------')
    model = Classifier(x_test.shape[1], 2)
    model.adv_weights(pattern, -taus[i].detach())
    
    torch.set_num_threads(1) # Required for multiprocessing 
    
    
    with multiprocessing.Pool(processes=NUM_PROCESS) as pool:
        x_tnr = list(tqdm(pool.imap_unordered(task_tnr, range(times), chunksize=2), total=times))
        x_tpr = list(tqdm(pool.imap_unordered(task_tpr, range(times), chunksize=2), total=times))
        
    torch.set_num_threads(72) # Required for multiprocessing 
    
    tpr = []
    for x in tqdm(x_tpr):
        out = model(x)[0]
        if torch.sum(out > 0) > 0:
            tpr.append(1)
        else:
            tpr.append(0)

    tnr = []
    for x in tqdm(x_tnr):
        out = model(x)[0]
        if torch.sum(out > 0) == 0:
            tnr.append(1)
        else:
            tnr.append(0)
    
    for i in range(runs):
        start_index = i*games
        end_index = (i+1)*games

        tp = sum(tpr[start_index:end_index])/games
        tn = sum(tnr[start_index:end_index])/games
        adv = tp/2 + tn/2
        
        report = {'eps' : eps,
                'adv': adv}
        
        result_log = pd.concat([result_log, pd.DataFrame.from_records([report])])
        with open(log_filename, 'wb') as logfile:
            pickle.dump(result_log, logfile)
            
    print(f'tpr = {np.mean(tpr)}')
    print(f'tnr = {np.mean(tnr)}')
    print(f'adv = {np.mean(tpr)/2 + np.mean(tnr)/2}')

eps=6 --------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 143.97it/s]

tpr = 1.0
tnr = 0.99
adv = 0.995
